In [ ]:
from zipfile import ZipFile
zip = ZipFile('Resources/archive.zip')
zip.extractall('Resources')

In [ ]:
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
#Display all the columns (to see which to drop)
pd.set_option("display.max_columns", None)

# Read in Mutual Fund informaiton csv


In [ ]:
#Read in CSV data 
#MutualFunds
mutualFunds= pd.read_csv(
    Path("Resources/MutualFunds.csv")
,index_col="fund_symbol")
mutualFunds.head()

In [ ]:
# Find columns with only 1 value to drop
mutualFunds.loc[: , mutualFunds.dtypes== "object"].nunique()

In [ ]:
#convert dates to date
mutualFunds["inception_date"] = pd.to_datetime(mutualFunds["inception_date"])
mutualFunds["management_start_date"] = pd.to_datetime(mutualFunds["management_start_date"])
mutualFunds["returns_as_of_date"] = pd.to_datetime(mutualFunds["returns_as_of_date"])

In [ ]:
mutualFunds_counts= mutualFunds.loc[: ,mutualFunds.dtypes=="object"].nunique()
mutualFunds_counts_one= mutualFunds_counts[mutualFunds_counts == 1].index.to_list()
print(mutualFunds_counts_one)

In [ ]:
#Drop columns with only 1 unique value
mutualFunds.drop(columns=mutualFunds_counts_one,inplace=True)
mutualFunds

_We noticed that there are many `NaN` values throughout the data set. We will now explore which columns/rows have NaN and will remove an appropriate amount of columns/rows._

In [ ]:
mutualFunds.dropna(axis = 0)

If we drop rows that have an `NaN`, then all data is removed. Instead, let's look at the columns that have missing data.

In [ ]:
# Looking at each column and the count of NaN in each column
list = []
for index, row in pd.DataFrame(mutualFunds.isna().sum()).iterrows():
    list.append((index,row))


In [ ]:
# Handling Missing Values
# Fill missing values in numerical columns with mean or median
numerical_cols = mutualFunds.select_dtypes(include=['number']).columns
mutualFunds[numerical_cols] = mutualFunds[numerical_cols].fillna(mutualFunds[numerical_cols].mean())

# Fill missing values in categorical columns with mode
categorical_cols = mutualFunds.select_dtypes(include=['object']).columns
mutualFunds[categorical_cols] = mutualFunds[categorical_cols].fillna(mutualFunds[categorical_cols].mode().iloc[0])

In [ ]:
# Encoding Categorical Variables
# One-hot encode categorical variables
mutualFunds_encoded = pd.get_dummies(mutualFunds, columns=categorical_cols)

# Scaling Numerical Features
# Separate numerical columns for scaling
numerical_cols = mutualFunds_encoded.select_dtypes(include=['number']).columns

In [ ]:
list[0]

We plan to find the number of columns that have above a certain percentage of `NaN` and then remove those columns. We have chosen 60% so far.

In [ ]:
threshold = 0.6*len(mutualFunds)
drop_columns = []
for i in range(len(list)):
    if list[i][1][0] >= threshold:
        drop_columns.append(list[i][0])

In [ ]:
print(len(drop_columns))
drop_columns


In [ ]:
mutualFunds.drop(columns=drop_columns, inplace = True)
mutualFunds

In [ ]:
# Find columns with only 1 value to drop
mutualFunds.loc[: , mutualFunds.dtypes== "object"].nunique()

In [ ]:
#Drop unnecessary columns
mutualFunds.drop(columns=["fund_short_name", "fund_long_name","management_name", "management_bio", "investment_strategy"],inplace=True)


In [ ]:
#Remove quarterly data
mutualFunds.drop(columns=["fund_return_2021_q1",
"fund_return_2020_q4","fund_return_2020_q3","fund_return_2020_q2","fund_return_2020_q1",
"fund_return_2019_q4","fund_return_2019_q3","fund_return_2019_q2","fund_return_2019_q1",
"fund_return_2018_q4","fund_return_2018_q3","fund_return_2018_q2","fund_return_2018_q1",
"fund_return_2017_q4","fund_return_2017_q3","fund_return_2017_q2","fund_return_2017_q1",
"fund_return_2016_q4","fund_return_2016_q3","fund_return_2016_q2","fund_return_2016_q1",
"fund_return_2015_q4","fund_return_2015_q3","fund_return_2015_q2","fund_return_2015_q1",
"fund_return_2014_q4","fund_return_2014_q3","fund_return_2014_q2","fund_return_2014_q1",
"fund_return_2013_q4","fund_return_2013_q3","fund_return_2013_q2","fund_return_2013_q1",
"fund_return_2012_q4","fund_return_2012_q3","fund_return_2012_q2","fund_return_2012_q1",
"fund_return_2011_q4","fund_return_2011_q3","fund_return_2011_q2","fund_return_2011_q1",
"fund_return_2010_q4","fund_return_2010_q3","fund_return_2010_q2","fund_return_2010_q1",
"fund_return_2009_q4","fund_return_2009_q3","fund_return_2009_q2","fund_return_2009_q1",
"fund_return_2008_q3","fund_return_2008_q2","fund_return_2008_q1",
"fund_alpha_3years","fund_beta_3years","fund_mean_annual_return_3years","fund_r_squared_3years","fund_stdev_3years","fund_sharpe_ratio_3years","fund_treynor_ratio_3years",
"fund_alpha_5years","fund_beta_5years","fund_mean_annual_return_5years","fund_r_squared_5years","fund_stdev_5years","fund_sharpe_ratio_5years","fund_treynor_ratio_5years",
"fund_alpha_10years","fund_beta_10years","fund_mean_annual_return_10years","fund_r_squared_10years","fund_stdev_10years","fund_sharpe_ratio_10years","fund_treynor_ratio_10years",
"fund_return_category_rank_ytd","fund_return_category_rank_1month","fund_return_category_rank_3months","fund_return_category_rank_1year","fund_return_category_rank_3years",
"fund_return_category_rank_5years","load_adj_return_1year","load_adj_return_3years","load_adj_return_5years","load_adj_return_10years",
"top10_holdings"],inplace=True)

In [ ]:
#Remove metrics that we can caluclate ourselves
#keep: "esg_score","environment_score", "sustainability_score", "sustainability_rank",   "social_score",  "governance_score", 
mutualFunds.drop(columns=["esg_peer_count","peer_esg_min", "peer_esg_avg", "peer_esg_max",
"peer_environment_min", "peer_environment_avg", "peer_environment_max", 
"peer_social_min", "peer_social_avg", "peer_social_max",
"peer_governance_min", "peer_governance_avg", "peer_governance_max"],inplace=True)


In [ ]:
mutualFunds.loc[: , mutualFunds.dtypes== "object"].nunique()

Done with Data cleanup and preprocessing


In [ ]:
# Calculate the average for each column
column_means = mutualFunds.mean(skipna=True, numeric_only=True)
# Replace NaN values in each column with the respective column average
mutualFunds.fillna(column_means, inplace=True)

In [ ]:
mutualFunds.dropna(inplace= True)
mutualFunds

In [ ]:
mutualFunds.dtypes

# Binning!

In [ ]:
mutualFunds.loc[:,mutualFunds.dtypes == "object"].dtypes

Binning `fund_category`

In [ ]:
fund_category_type_count = mutualFunds["fund_category"].value_counts()[mutualFunds["fund_category"].value_counts() > 150]
print(100*fund_category_type_count.sum()/len(mutualFunds))
print(fund_category_type_count.count())

In [ ]:
for cat in mutualFunds["fund_category"]:
    if cat in fund_category_type_count.index.to_list():
        next
    else:
        mutualFunds["fund_category"] = mutualFunds["fund_category"].replace(cat, "Other")

In [ ]:
mutualFunds["fund_category"].value_counts()

Binning `fund_family_type_count`

In [ ]:
fund_family_type_count = mutualFunds["fund_family"].value_counts()[mutualFunds["fund_family"].value_counts() > 150]
print(100*fund_family_type_count.sum()/len(mutualFunds))
print(fund_family_type_count.count())

In [ ]:
for cat in mutualFunds["fund_family"]:
    if cat in fund_family_type_count.index.to_list():
        next
    else:
        mutualFunds["fund_family"] = mutualFunds["fund_family"].replace(cat, "Other")

In [ ]:
mutualFunds["fund_family"].value_counts()

Binning `esg_peer_group`

In [ ]:
#finding threshhold to bin esg_peer_group
esg_peer_type_count = mutualFunds["esg_peer_group"].value_counts()[mutualFunds["esg_peer_group"].value_counts() > 100]
print(100* esg_peer_type_count.sum()/len(mutualFunds))
print(esg_peer_type_count.count())

In [ ]:
#check the bins of esg_peer_group
esg_peer_type_count.index.to_list()
#replace bins in dataframe for esg_peer_group
for esg in mutualFunds["esg_peer_group"]:
    if esg in esg_peer_type_count.index.to_list():
        next
    else:
        mutualFunds["esg_peer_group"] = mutualFunds["esg_peer_group"].replace(esg, "Other")

In [ ]:
mutualFunds["esg_peer_group"].value_counts()

In [ ]:
mutualFunds

# SQLite Database 

In [ ]:
import os
import pandas as pd
import sqlite3
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
# Create directory for cleaned CSV if it doesn't exist
output_directory = "Resources/Cleaned"
os.makedirs(output_directory, exist_ok=True)


In [ ]:
# Save the cleaned CSV file
output_file_path = os.path.join(output_directory, "binned_mutual_funds.csv")
binned_mutual_funds.to_csv(output_file_path, header=True, index=True)

In [ ]:
# Connect to SQLite database
conn = sqlite3.connect('project_4.db')

# Insert data from CSV into the existing table
cleaned_data = pd.read_csv(output_file_path)
cleaned_data.to_sql('table_1', conn, if_exists='replace', index=False)

# Close the database connection
conn.close()

# Data Implementation 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Convert datetime columns in mutualFunds to numerical representation
mutualFunds['management_start_date'] = pd.to_datetime(mutualFunds['management_start_date'])
mutualFunds['inception_date'] = pd.to_datetime(mutualFunds['inception_date'])
mutualFunds['returns_as_of_date'] = pd.to_datetime(mutualFunds['returns_as_of_date'])

# Select features and target variable

In [ ]:
# Select features and target variable
features = ['fund_category', 'esg_score', 'fund_return_3years', 'inception_date', 'top10_holdings_total_assets']
target_variable = 'year_to_date_return'

# Create X and y
X = mutualFunds[features]
y = mutualFunds[target_variable]

# Preprocess data
X = pd.get_dummies(X)
X.fillna(X.mean(), inplace=True)
X['inception_date'] = (X['inception_date'] - X['inception_date'].min()).dt.days

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Select features and target variable
features = ['fund_category', 'esg_score', 'fund_return_3years', 'inception_date', 'top10_holdings_total_assets']
target_variable = 'year_to_date_return'

# Create X and y
X = mutualFunds[features]
y = mutualFunds[target_variable]

# Preprocess data
X = pd.get_dummies(X)
X.fillna(X.mean(), inplace=True)
X['inception_date'] = (X['inception_date'] - X['inception_date'].min()).dt.days

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Model Testing

In [ ]:
# Initialize and train the model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("Mean Squared Error:", mse)
print("R-squared:", r2)
